In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
BATCH_SIZE = 32
EVAL_BATCH_SIZE = 256
LEARNING_RATE = 0.0001

## Pre

In [ ]:
import os
from copy import deepcopy
from typing import Any, Dict, Tuple

import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
from datasets import load_dataset
from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer as PLTrainer
from pytorch_lightning import seed_everything
from torch import Tensor, nn
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, F1Score, MetricCollection, Precision, Recall
from transformers import (
    AdamW,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_constant_schedule_with_warmup,
)

from energizer import AccumulatorStrategy, RandomStrategy, Trainer
from energizer.acquisition_functions import entropy, expected_entropy

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# renames "label" to "labels"
collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")

# load dataset
dataset = load_dataset("pietrolesci/ag_news", "concat")

# tokenize
dataset = dataset.map(lambda ex: tokenizer(ex["text"]), batched=True)
columns_to_keep = ["label", "input_ids", "token_type_ids", "attention_mask"]

# train-val split and record datasets
train_set, test_set = dataset["train"], dataset["test"]
_split = train_set.train_test_split(0.3)
train_set, val_set = _split["train"], _split["test"]

labels = train_set.features["label"].names
num_classes = len(labels)

# create dataloaders
batch_size = BATCH_SIZE
eval_batch_size = EVAL_BATCH_SIZE  # this is use when evaluating on the pool too
train_dl = DataLoader(
    train_set.with_format(columns=columns_to_keep),
    batch_size=batch_size,
    collate_fn=collator,
    num_workers=2,
)
val_dl = DataLoader(
    val_set.with_format(columns=columns_to_keep),
    batch_size=eval_batch_size,
    collate_fn=collator,
    num_workers=2,
)
test_dl = DataLoader(
    test_set.with_format(columns=columns_to_keep),
    batch_size=eval_batch_size,
    collate_fn=collator,
    num_workers=2,
)

In [ ]:
class TransformerModel(LightningModule):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        learning_rate: float = 0.00001,
        num_warmup_steps: int = 50,
    ) -> None:
        super().__init__()
        self.model_name = model_name
        self.num_classes = num_classes
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_classes,
        )
        self.learning_rate = learning_rate
        self.num_warmup_steps = num_warmup_steps
        for stage in ("train", "val", "test"):
            metrics = MetricCollection(
                {
                    "accuracy": Accuracy(),
                    "precision_macro": Precision(num_classes=num_classes, average="macro"),
                    "precision_micro": Precision(num_classes=num_classes, average="micro"),
                    "recall_macro": Recall(num_classes=num_classes, average="macro"),
                    "recall_micro": Recall(num_classes=num_classes, average="micro"),
                    "f1_macro": F1Score(num_classes=num_classes, average="macro"),
                    "f1_micro": F1Score(num_classes=num_classes, average="micro"),
                }
            )
            setattr(self, f"{stage}_metrics", metrics)

    def common_step(self, batch: Any, stage: str):
        """Outputs loss and logits, logs loss and metrics."""
        out = self(batch)
        logits, loss = out.logits, out.loss
        self.log(f"{stage}/loss", loss)

        metrics = getattr(self, f"{stage}_metrics")(logits, batch["labels"])
        self.log_dict(metrics)

        return loss

    def forward(self, batch) -> torch.Tensor:
        return self.model(**batch)

    def training_step(self, batch: Any, batch_idx: int = 0, optimizer_idx: int = 0) -> Dict[str, Any]:
        return self.common_step(batch, "train")

    def validation_step(self, batch: Any, batch_idx: int = 0) -> Dict[str, Any]:
        return self.common_step(batch, "val")

    def test_step(self, batch: Any, batch_idx: int = 0) -> Dict[str, Any]:
        return self.common_step(batch, "test")

    def configure_optimizers(self) -> Dict[str, Any]:
        optimizer = AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=self.learning_rate)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": get_constant_schedule_with_warmup(
                    optimizer=optimizer, num_warmup_steps=self.num_warmup_steps
                ),
                "monitor": "val/loss",
                "frequency": 1,
                "interval": "step",
            },
        }

In [ ]:
class EntropyStrategy(AccumulatorStrategy):
    """A implememntation of the `Entropy` active learning strategy."""

    def pool_step(self, batch: Dict[str, Tensor], batch_idx: int) -> Tensor:
        batch.pop("labels")
        logits = self(batch).logits
        return entropy(logits)

In [ ]:
# entropy_strategy = EntropyStrategy(deepcopy(model))
# random_strategy = RandomStrategy(deepcopy(model))

# batch = next(iter(train_dl))
# model(batch).logits.shape, entropy_strategy(batch).logits.shape, random_strategy(batch).logits.shape

## Active fit

In [ ]:
model = TransformerModel(model_name=MODEL_NAME, num_classes=num_classes, learning_rate=LEARNING_RATE)

### Random strategy

In [ ]:
random_strategy = RandomStrategy(deepcopy(model))

seed_everything(1994)

trainer = Trainer(
    query_size=50,
    max_epochs=3,
    max_labelling_epochs=10,
    # total_budget=5,
    test_after_labelling=True,
    accelerator="gpu",
    # for testing purposes
    # limit_train_batches=10,
    limit_val_batches=1,
    # limit_test_batches=10,
    # limit_pool_batches=10,
    # log_every_n_steps=1,
)

results = trainer.active_fit(
    model=random_strategy,
    train_dataloaders=train_dl,
    val_dataloaders=val_dl,
    test_dataloaders=test_dl,
)

In [ ]:
random_df = pd.DataFrame(
    data=[(l.data_stats["train_size"], *l.test_outputs[0].values()) for l in results],
    columns=("train_size", *results[0].test_outputs[0].keys()),
)
random_df

In [ ]:
entropy_strategy = EntropyStrategy(deepcopy(model))

seed_everything(1994)

trainer = Trainer(
    query_size=50,
    max_epochs=3,
    max_labelling_epochs=2,
    # total_budget=5,
    test_after_labelling=True,
    accelerator="gpu",
    # for testing purposes
    # limit_train_batches=10,
    limit_val_batches=1,
    # limit_test_batches=10,
    # limit_pool_batches=10,
    # log_every_n_steps=1,
)

results = trainer.active_fit(
    model=entropy_strategy,
    train_dataloaders=train_dl,
    val_dataloaders=val_dl,
    test_dataloaders=test_dl,
)

In [ ]:
entropy_df = pd.DataFrame(
    data=[(l.data_stats["train_size"], *l.test_outputs[0].values()) for l in results],
    columns=("train_size", *results[0].test_outputs[0].keys()),
)
entropy_df

In [ ]:
random_df